# Transfer Learning - InceptionV3+FC


In [ ]:
# option to view all the outputs of a cell and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

dataset_dir = '/content/drive/My Drive/NeuralNetwork_project/MaskDataset'

In [ ]:
# the original file was implemented in Google Colab to take advantage of the free GPU 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# img shape (use those of the original architecture)
img_h = 299
img_w = 299

# import the architecture 
from keras.applications.inception_v3 import preprocess_input 
base = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

87916544/87910968 [==============================] - 0s 0us/step


In [ ]:
# labels of the training set images are provided by a text file that matches each image to the corresponding class
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)
df = pd.DataFrame(dic.items())
df.rename(columns = {0:'filename', 1:'class'}, inplace = True)
df['class'] = df['class'].astype(str)

In [ ]:
# divide the dataset into training set and validation set, reserving 80% of the data for the training set
# and 20% for the validation set (the total images in the dataset are 5600)
df_shuffled = df.sample(frac=1, random_state=SEED).reset_index()
df_train = df_shuffled.iloc[0:4500]
df_valid = df_shuffled.iloc[4500:]

# using stratified sampling is not necessary as the split already produces a balanced division, 
# as we can see in the following:
df_valid["class"].value_counts() 

2    374
0    373
1    367
Name: class, dtype: int64

In [ ]:
# set the data augmentation parameters
# (they are chosen in a way that modify the images as much as possible, without completely altering them)

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        brightness_range=(0.8,1.2),
                                        shear_range=10,
                                        channel_shift_range=150,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        preprocessing_function=preprocess_input)  
else:
    train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# Batch size
bs = 32

num_classes=3

training_dir = os.path.join(dataset_dir,"training")

# Training
train_gen = train_data_gen.flow_from_dataframe(df_train,
                                               training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               target_size=(img_h,img_w),
                                               seed=SEED)  

# Validation
valid_gen = valid_data_gen.flow_from_dataframe(df_valid,
                                               training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               target_size=(img_h,img_w),
                                               seed=SEED)


Found 4500 validated image filenames belonging to 3 classes.
Found 1114 validated image filenames belonging to 3 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()


In [ ]:
# Create Model 
# ------------
base.trainable = False

inputs = tf.keras.Input(shape=(img_h, img_w, 3))
x = inputs
x = base(x, training=False) 
# the "training = False" parameter allows to keep the batch normalization layers freezed, 
# even when the others will be made trainable
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
# add a FC layer with l2 regularization
x = tf.keras.layers.Dense(units=1024, activation='relu', kernel_regularizer='l2')(x)
# add a dropout layer to further control overfitting
x = tf.keras.layers.Dropout(0.2, name="top_dropout")(x)
# final softmax for classification
outputs = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Visualize created model as a table
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
top_dropout (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 23,904,035
Trainable params: 2,101,251
Non-trainable params: 21,802,784
__________________________________

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# as recommended by Keras documentation, I do the first training keeping the base freezed
model.fit(x=train_dataset,
          epochs=15, 
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen))

Epoch 1/15
141/141 [==============================] - 1168s 8s/step - loss: 12.4636 - accuracy: 0.5773 - val_loss: 10.3878 - val_accuracy: 0.6777
Epoch 2/15
141/141 [==============================] - 152s 1s/step - loss: 9.0899 - accuracy: 0.6360 - val_loss: 7.7313 - val_accuracy: 0.6912
Epoch 3/15
141/141 [==============================] - 151s 1s/step - loss: 6.9015 - accuracy: 0.6520 - val_loss: 5.9887 - val_accuracy: 0.6822
Epoch 4/15
141/141 [==============================] - 152s 1s/step - loss: 5.3943 - accuracy: 0.6671 - val_loss: 4.7305 - val_accuracy: 0.7065
Epoch 5/15
141/141 [==============================] - 150s 1s/step - loss: 4.3486 - accuracy: 0.6727 - val_loss: 3.8515 - val_accuracy: 0.7047
Epoch 6/15
141/141 [==============================] - 149s 1s/step - loss: 3.5729 - accuracy: 0.6802 - val_loss: 3.2074 - val_accuracy: 0.7226
Epoch 7/15
141/141 [==============================] - 149s 1s/step - loss: 3.0075 - accuracy: 0.6784 - val_loss: 2.6961 - val_accuracy: 0.7

Now I proceed with fine tuning by unfreezing the base. I can decide whether to unlock all the layers of the base or just some. After trying both options, I found that unlocking all layers led to the best results

In [ ]:
#unfreeze only some layers 
# ------------
#base.trainable = True
#unfreeze=False
#for layer in base.layers:
#  layer.trainable=False
#  if unfreeze and not (isinstance(layer, tf.keras.layers.BatchNormalization)):
#    layer.trainable=True
#  if layer.name=='mixed1': #it is possible to select also mixed2,3,4 and so on...
#    unfreeze=True

#model.compile(loss=loss, metrics=metrics, optimizer=tf.keras.optimizers.Adam(1e-4))
#model.summary()

In [ ]:
#unfreeze all the layers
base.trainable = True
model.compile(loss=loss, metrics=metrics, optimizer=tf.keras.optimizers.Adam(1e-4))
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
top_dropout (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 23,904,035
Trainable params: 23,869,603
Non-trainable params: 34,432
_____________________________________

In [ ]:
# plot to view the progress of the training
%load_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/NeuralNetwork_project/classification_experiments/ --port 6009

In [ ]:
# set the automatic saving checkpoints

import os

cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/NeuralNetwork_project/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

model_name = 'InceptionV3'

exp_dir = os.path.join(exps_dir, model_name)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=False)  

callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [ ]:
# final training
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/100
141/141 [==============================] - ETA: 0s - loss: 1.2581 - accuracy: 0.5967WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/NeuralNetwork_project/classification_experiments/InceptionV3/ckpts/cp_01.ckpt/assets
141/141 [==============================] - 222s 2s/step - loss: 1.2581 - accuracy: 0.5967 - val_loss: 0.8519 - val_accuracy: 0.7648
Epoch 2/100
141/141 [==============================] - 222s 2s/step - loss: 0.7281 - accuracy: 0.8173 - val_loss: 0.5687 - val_accuracy: 0.8851
Epoch 3/100


In [ ]:
# save the model
model.save('/content/drive/My Drive/NeuralNetwork_project/classification_experiments/inceptionV3') 

INFO:tensorflow:Assets written to: /content/drive/My Drive/NeuralNetwork_project/classification_experiments/inceptionV3/assets


In [ ]:
# create the csv file for the submission on Kaggle
import os

def create_csv(results, results_dir='/content/drive/My Drive/NeuralNetwork_project'):

    csv_fname = 'results_InceptionV3.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
# ....

test_dir = '/content/drive/My Drive/NeuralNetwork_project/MaskDataset/test/'
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
   img = Image.open(test_dir + image_name).convert('RGB')
   img = img.resize((img_h,img_w))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   img_array = preprocess_input(img_array)
   softmax = model.predict(img_array)
   prediction = np.argmax(softmax)  
   results[image_name] = prediction

In [ ]:
create_csv(results)